In [5]:
import pandas as pd
patch = pd.read_csv("patch.csv")
patch.head()

,val_rmse_mean,val_rmse_std,val_mae_mean,val_r2_mean,train_rmse_mean,train_mae_mean,train_r2_mean,n_folds,n_samples,model,scheme,level,atom_index,note
0,0.188953,0.053726,0.168094,-3.838650,0.1892,0.167383,-3.558292,5,510,FlaxMLP,ADCH,hf,0,NaN
1,0.035877,0.005584,0.013107,0.841620,NaN,NaN,NaN,5,510,KNN,ADCH,hf,0,NaN
2,0.065052,0.003709,0.056129,0.481692,NaN,NaN,NaN,5,510,SVR,ADCH,hf,0,NaN
3,0.008250,0.004082,0.004830,0.989646,NaN,NaN,NaN,5,510,DecisionTree,ADCH,hf,0,NaN
4,0.005610,0.002466,0.002709,0.995190,NaN,NaN,NaN,5,510,RandomForest,ADCH,hf,0,NaN


In [22]:
charges_long = pd.read_csv("df_charges_long.csv")
charges_long.head()

,r1,r2,ang,level,atom_index,atom,scheme,value
0,1.0,1.0,120.0,hf,0,C,Hirshfeld,0.424593
1,1.0,1.0,120.0,hf,1,O,Hirshfeld,-0.212303
2,1.0,1.0,120.0,hf,2,O,Hirshfeld,-0.212289
3,1.0,1.0,120.0,hf,0,C,VDD,0.430338
4,1.0,1.0,120.0,hf,1,O,VDD,-0.217065


In [24]:
charges_long.groupby(["scheme", ]).describe()["value"]

,count,mean,std,min,25%,50%,75%,max
scheme,,,,,,,,
ADCH,3060.0,-8.823541e-13,0.496034,-0.504493,-0.408955,-0.243451,0.630342,0.783232
Becke,3060.0,-3.594765e-13,0.508179,-0.566010,-0.416316,-0.264895,0.652800,0.866921
CHELPG,3060.0,-3.267982e-13,0.630981,-0.575979,-0.517867,-0.354911,0.791134,1.092346
CM5,3060.0,-3.267954e-14,0.368915,-0.383262,-0.284738,-0.192725,0.506306,0.550493
Hirshfeld,3060.0,-9.150327e-13,0.300153,-0.363132,-0.231989,-0.129775,0.407424,0.500805
MBIS,3060.0,1.233581e-18,0.854387,-0.734478,-0.671170,-0.526293,1.101061,1.468953
MBIS_raw,3060.0,4.491212e-06,0.854386,-0.734467,-0.671173,-0.526284,1.101069,1.468958
MK,3060.0,-3.921578e-13,0.597194,-0.561811,-0.496355,-0.322669,0.741604,1.060563
VDD,3060.0,-1.568628e-12,0.330490,-0.405693,-0.262468,-0.134326,0.438735,0.574003


In [25]:
(patch[patch["scheme"]=="MBIS_raw"]).groupby(["scheme", "model", ]).describe()["val_rmse_mean"].sort_values("mean")

count      mean       std       min       25%  \
scheme   model                                                             
MBIS_raw GradientBoosting    6.0  0.006306  0.001930  0.003869  0.004774   
         RandomForest        6.0  0.007239  0.002307  0.004397  0.005511   
         KNN                 6.0  0.009039  0.003611  0.004628  0.007440   
         DecisionTree        6.0  0.011563  0.005121  0.007293  0.007743   
         SVR                 6.0  0.056105  0.005724  0.049100  0.052007   
         FlaxMLP             6.0  0.211497  0.072655  0.125282  0.152770   

                                50%       75%       max  
scheme   model                                           
MBIS_raw GradientBoosting  0.006524  0.007603  0.008767  
         RandomForest      0.007151  0.009048  0.010089  
         KNN               0.008598  0.009587  0.015438  
         DecisionTree      0.009299  0.015965  0.018079  
         SVR               0.056001  0.059159  0.064641  
         FlaxMLP           0.224065  0.243462  0.316639